In [34]:
import folium
import geopandas as gpd
from shapely.ops import linemerge

# 경로 데이터 (노드 ID 리스트)
route = [
    28000275483,
    28000276046,
    28000275496,
    28000502970,
    28000502968,
    28000502967,
    28000471964,
    28000276048,
    28000122204,
    28000040470,
    28000040701,
    28000040716,
    28000039872,
    28000819499,
    28000819503
]

# 네트워크 데이터 로드
network_gpkg = "../data/network/map.gpkg"
network = gpd.read_file(network_gpkg, layer='link')
node = gpd.read_file(network_gpkg, layer='node')
print(f"네트워크 데이터 로드 완료, 데이터 수: {len(network)}")


네트워크 데이터 로드 완료, 데이터 수: 424271


# 데이터 변환


In [31]:
import pandas as pd
# 기존 데이터프레임의 's_node_id'와 'e_node_id'를 서로 바꾼 새로운 데이터프레임 생성
swapped_network = network.copy()  # 데이터프레임 복사
swapped_network['s_node_id'], swapped_network['e_node_id'] = network['e_node_id'], network['s_node_id']  # 컬럼 값 교환

# 원본 데이터프레임과 수정된 데이터프레임을 합침
network_doubled = pd.concat([network, swapped_network], ignore_index=True)

# s_node_id 컬럼의 데이터 타입 확인
network_doubled['s_node_id'] = pd.to_numeric(network_doubled['s_node_id'], errors='coerce')

print(network_doubled['s_node_id'].dtype)
print(type(route[1]))
network_doubled

int64
<class 'int'>


,limit_,danji_id,length,e_node_id,emd_nm,link_id,sgg_nm,sido_nm,link_cate,link_code,...,s_node_id,rd_nm,link_name,edit_date,floor,SHAPE__Length,edit_code,objectid,wroad_cate,geometry
0,N,NaN,22.26,28000443410,청라2동,28300152678,서구,인천광역시,12,31202,...,28000443409,None,보행자연결시설_점자블록_보행로,2021-12-03,None,22.263237,A,1,3,"LINESTRING (126.64632 37.53902, 126.64632 37.5..."
1,N,NaN,12.05,28000506095,송도3동,28300394332,연수구,인천광역시,12,31202,...,28000421420,None,보행자연결시설_점자블록_보행로,2021-12-03,None,12.048084,A,2,3,"LINESTRING (126.66730 37.38054, 126.66731 37.3..."
2,N,NaN,82.59,28000159228,길상면,28200036232,강화군,인천광역시,05,20501,...,28100159227,길상로,단방향도로_일반도로_보차혼용도로,2021-12-03,None,82.589296,A,3,2,"LINESTRING (126.50016 37.63340, 126.50017 37.6..."
3,N,NaN,123.11,28000159230,길상면,28200036270,강화군,인천광역시,05,20502,...,28000092384,길상로,단방향도로_일반도로_보행로,2021-12-03,None,123.112486,A,4,2,"LINESTRING (126.49513 37.63369, 126.49514 37.6..."
4,N,NaN,145.02,28000117221,길상면,28200036338,강화군,인천광역시,05,20501,...,28000159236,None,단방향도로_일반도로_보차혼용도로,2021-12-03,None,145.022940,A,5,2,"LINESTRING (126.52485 37.63538, 126.52474 37.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848537,N,1218.0,14.03,28000261036,운서동,28200426937,중구,인천광역시,09,20902,...,28000848424,None,단방향도로_단지내_보행로,2021-12-03,None,14.027365,A,424267,2,"LINESTRING (126.48319 37.44679, 126.48329 37.4..."
848538,N,1218.0,14.86,28000848424,운서동,28200426938,중구,인천광역시,09,20902,...,28100848426,None,단방향도로_단지내_보행로,2021-12-03,None,14.863330,A,424268,2,"LINESTRING (126.48329 37.44689, 126.48313 37.4..."
848539,N,1218.0,2.89,28000848424,운서동,28200426939,중구,인천광역시,09,20902,...,28000848428,None,단방향도로_단지내_보행로,2021-12-03,None,2.887314,A,424269,2,"LINESTRING (126.48329 37.44689, 126.48332 37.4..."
848540,N,1218.0,2.72,28000848428,운서동,28200426940,중구,인천광역시,09,20902,...,28100848430,None,단방향도로_단지내_보행로,2021-12-03,None,2.715641,A,424270,2,"LINESTRING (126.48332 37.44690, 126.48333 37.4..."


In [32]:
# 중복된 행 찾기 (모든 열을 기준으로)
duplicates = network_doubled.duplicated(keep=False)

# 중복된 행이 있는지 확인
if duplicates.any():
    print("중복된 행이 있습니다.")
    # 중복된 행 출력
    print(network_doubled[duplicates])
else:
    print("중복된 행이 없습니다.")

중복된 행이 없습니다.


In [33]:
# 엣지 데이터를 같은 GeoPackage 파일에 다른 레이어로 저장
network_doubled.to_file('C:/moduroad/data/network/map.gpkg', layer='link', driver='GPKG')

# 시각화

In [2]:
import osmnx as ox
import networkx as nx
import pandas as pd
import math
import numpy as np
from shapely.geometry import LineString, Point
from shapely.ops import linemerge
from shapely.geometry import MultiLineString
import pickle
import folium

def find_shortest_path(start, end, network, speed_kmph=3.7):
    """
    start: 시작점 좌표 (위도, 경도)
    end: 도착점 좌표 (위도, 경도)
    network: 네트워크 그래프
    speed_kmph: 평균 이동 속도(km/h)
    """
    G = network
    G.graph['crs'] = 'epsg:4326'
   
    # 가장 가까운 노드 찾기
    start_node = ox.distance.nearest_nodes(G, start[1], start[0])
    end_node = ox.distance.nearest_nodes(G, end[1], end[0])
    
    # A* 알고리즘으로 최단 경로 찾기
    route = nx.astar_path(G, start_node, end_node, weight='length')  # 여기서 'weight'를 'length'로 수정해야 합니다.
    
    # 라인 출력 및 거리, 시간 계산
    line_strings = []
    total_length = 0  # 총 이동 거리
    total_time = 0  # 총 이동 시간
    
    for i in range(len(route)-1):
        source = route[i]
        target = route[i+1]
        edge_data = G.get_edge_data(source, target, 0)
        
        if edge_data and 'geometry' in edge_data:
            line_strings.append(edge_data['geometry'])
            edge_length = edge_data['length']  # 엣지 길이
            total_length += edge_length
        else:
            print(f"경로상의 노드 쌍 {source} -> {target} 사이의 엣지를 찾을 수 없습니다.")
    
    # 시간 계산 (시간 = 거리 / 속도)
    # 속도를 m/s로 변환
    speed_mps = (speed_kmph * 1000) / 3600
    total_time = (total_length / speed_mps) //60
    
    # 모든 LineString 객체들을 하나로 합침
    combined_line_string = linemerge(line_strings)

    if isinstance(combined_line_string, MultiLineString):
        line_list = [list(line.coords) for line in combined_line_string.geoms]
        flat_list = [item for sublist in line_list for item in sublist]
    else:
        line_list = list(combined_line_string.coords)
        flat_list = line_list
        print("리스트")
        print(flat_list)

    if total_length >= 1000:
        total_length = f"{total_length / 1000:.2f}km"
    else:
        total_length = f"{total_length}m"
    
    total_time = f"{total_time:.0f}분"




    return combined_line_string, total_length, total_time

    

#--------------------------------------------------------------------------------------------------------------------#
#test

file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'
with open(file_path, 'rb') as file:
    network_g = pickle.load(file)
    
start = (37.379625,126.619987)
end = (37.397111,126.632043)    
combined_line_string, total_length, total_time = find_shortest_path(start, end, network_g)

print(f"총 거리 : {total_length}\n소요 시간: {total_time}")
# Folium 지도 생성
m = folium.Map(location=[37.37463, 126.64481], zoom_start=14)

# 합쳐진 LineString을 Folium 지도에 추가 (비어 있지 않은 경우에만 추가)
if not combined_line_string.is_empty:
    folium.GeoJson(combined_line_string, style_function=lambda x: {'color': 'blue', 'weight': 5}).add_to(m)
else:
    print("지도에 추가할 LineString이 없습니다.")

# 시작 지점에 마커 추가
folium.Marker(
    location=start,
    popup="Start",
    icon=folium.Icon(color='green')
).add_to(m)

# 종료 지점에 마커 추가
folium.Marker(
    location=end,
    popup="End",
    icon=folium.Icon(color='red')
).add_to(m)
# 지도를 HTML 파일로 저장
m


리스트
[(126.61980583548866, 37.380389948801074), (126.62002756001654, 37.380792274907556), (126.62009952302223, 37.38089418135401), (126.62018024118994, 37.38098048304224), (126.62030228506333, 37.38108080709331), (126.62051056722032, 37.381224658386174), (126.62061083581565, 37.381281940704795), (126.62077415279771, 37.38135481410263), (126.62083360880538, 37.38139040235958), (126.62090367514529, 37.38138714800418), (126.62127951905595, 37.3814630310391), (126.62195599952858, 37.38167378970945), (126.62215965438925, 37.38172643738442), (126.62235142199003, 37.38176514708759), (126.62326321574069, 37.38188347480059), (126.62349740936331, 37.38189442996745), (126.62374004242483, 37.38188509037559), (126.62379885955686, 37.381896456829814), (126.62386665056079, 37.38185075571744), (126.62399950992662, 37.38173336887701), (126.62414452072133, 37.38165140728685), (126.6242634293544, 37.38160446036244), (126.62477921529103, 37.38143296816375), (126.62492877426821, 37.38140762751352), (126.625

# G 생성


In [35]:
import geopandas as gpd
import networkx as nx

# network_gpkg 파일 로드
nodes_gdf = node
links_gdf = network

# 멀티그래프 생성
G = nx.MultiGraph()

# 노드 추가
for idx, row in nodes_gdf.iterrows():
    G.add_node(row['node_id'], x=row['longitude'],y=row['latitude'])

# 엣지 추가
for idx, row in links_gdf.iterrows():
    G.add_edge(row['s_node_id'], row['e_node_id'], link_id=row['link_id'], 
               weight=row['SHAPE__Length'], geometry=row['geometry'])

# 그래프 생성 확인
print(f"노드 수: {G.number_of_nodes()}")
print(f"엣지 수: {G.number_of_edges()}")


노드 수: 322251
엣지 수: 424271


In [38]:
# 그래프의 기본 정보 출력
print("그래프 정보:")
print(f"노드 수: {G.number_of_nodes()}")
print(f"엣지 수: {G.number_of_edges()}")
print()

# 첫 번째 노드의 정보 및 데이터 타입 출력
first_node = list(G.nodes(data=True))[55350]
print(first_node)
print(f"노드 ID: {first_node[0]}, 데이터 타입: {type(first_node[0])}")
print(f"속성: {first_node[1]}, 데이터 타입: {type(first_node[1]['x'])}")
print()

# 첫 번째 엣지의 정보 및 데이터 타입 출력
first_edge = list(G.edges(data=True))[0]
print(first_edge)
print(f"엣지 시작: {first_edge[0]}, 데이터 타입: {type(first_edge[0])}")
print(f"엣지 끝: {first_edge[1]}, 데이터 타입: {type(first_edge[1])}")
print(f"속성: {first_edge[2]}, 데이터 타입: {type(first_edge[2])}")
num = 0
nodes_to_remove = []  # 삭제할 노드를 저장할 리스트

for node, data in G.nodes(data=True):
    if 'x' not in data or 'y' not in data:
        nodes_to_remove.append(node)  # 삭제할 노드 추가
        num += 1

# 노드 삭제
G.remove_nodes_from(nodes_to_remove)

print(f"삭제된 노드 수: {num}")



그래프 정보:
노드 수: 322251
엣지 수: 424271

('28000072285', {'x': 126.72599, 'y': 37.508692})
노드 ID: 28000072285, 데이터 타입: <class 'str'>
속성: {'x': 126.72599, 'y': 37.508692}, 데이터 타입: <class 'float'>

('28100005236', '28000005235', {'link_id': '28200000067', 'weight': 14.02555745870837, 'geometry': <LINESTRING (126.369 37.463, 126.369 37.463)>})
엣지 시작: 28100005236, 데이터 타입: <class 'str'>
엣지 끝: 28000005235, 데이터 타입: <class 'str'>
속성: {'link_id': '28200000067', 'weight': 14.02555745870837, 'geometry': <LINESTRING (126.369 37.463, 126.369 37.463)>}, 데이터 타입: <class 'dict'>
삭제된 노드 수: 1


In [19]:
import networkx as nx
import pickle
file_path = 'C:/my_code/moduroad/data/network/api_graph.pickle'
with open(file_path, 'rb') as file:
    G = pickle.load(file)
# 변경된 그래프 구조 중 첫 번째 노드와 그 데이터만 출력
# 변경된 그래프 구조 중 첫 번째 노드와 그 데이터만 출력
print(next(iter(G.nodes(data=True))))
print(next(iter(G.edges(data=True))))

for node_id, data in G.nodes(data=True):
    # 기존 속성 존재 여부 확인 후 새로운 속성 할당 및 기존 속성 삭제
    if 'latitude' in data and 'longitude' in data:
        lat = data.pop('latitude')  # 기존 속성 삭제하면서 값을 가져옴
        lon = data.pop('longitude')  # 기존 속성 삭제하면서 값을 가져옴
        G.nodes[node_id]['y'] = lat  # 새로운 속성 이름으로 할당
        G.nodes[node_id]['x'] = lon  # 새로운 속성 이름으로 할당


print(next(iter(G.nodes(data=True))))




Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001DA5A6C4CD0>>
Traceback (most recent call last):
  File "c:\my_code\moduroad\venv_modu\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


('28100005236', {'y': 37.462942, 'x': 126.369205})
('28100005236', 28000005235, {'link_id': '28200000067', 'weight': 14.02555745870837, 'geometry': <LINESTRING (126.369 37.463, 126.369 37.463)>})
('28100005236', {'y': 37.462942, 'x': 126.369205})


In [39]:
import os
import pickle
with open('../data/network/api_graph.pickle', 'wb') as f:
    pickle.dump(G, f)
